In [1]:
import sys
import os
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Aggiunge il percorso relativo per la cartella `config`
sys.path.append(os.path.abspath("../scripts"))
# Aggiunge il percorso relativo per la cartella `scripts`
sys.path.append(os.path.abspath("."))

# DIR = os.path.dirname(os.path.abspath(__file__))


from configs.test_config import TestConfig
from network_definition import NetworkDefinition
from pypsa2smspp.transformation import Transformation
from datetime import datetime
import pysmspp

from pypsa2smspp.network_correction import (
    clean_marginal_cost,
    clean_global_constraints,
    clean_e_sum,
    clean_efficiency_link,
    clean_ciclicity_storage,
    clean_marginal_cost_intermittent,
    clean_storage_units,
    clean_stores,
    parse_txt_file,
    compare_networks,
    add_slack_unit
    )

def get_datafile(fname):
    return os.path.join(os.path.dirname(__file__), "test_data", fname)

#%% Network definition with PyPSA
config = TestConfig()
nd = NetworkDefinition(config)

nd.n = add_slack_unit(nd.n)
nd.n = clean_ciclicity_storage(nd.n)


network = nd.n.copy()
network.optimize(solver_name='gurobi')

# network.export_to_netcdf("test_pypsa.nc")

# network.model.to_file(fn = "f.lp")
#%% Transformation class
then = datetime.now()
transformation = Transformation(network, merge_links=True)
print(f"La classe di trasformazione ci mette {datetime.now() - then} secondi")

tran = transformation.convert_to_blocks()

WARNING    2025-09-17 15:08:43,945 pypsa.network.io _import_components_from_df  1732: The following Generator have buses which are not defined:
Index(['IT0 0 0 solar'], dtype='object')
WARNING    2025-09-17 15:08:44,020 pypsa.network.io _import_components_from_df  1732: The following Store have buses which are not defined:
Index(['IT0 0 H2'], dtype='object')
WARNING    2025-09-17 15:08:44,154 pypsa.consistency _log_or_raise  40  : The following buses have carriers which are not defined:
Index(['IT0 0 H2'], dtype='object', name='Bus')
WARNING    2025-09-17 15:08:44,178 pypsa.consistency _log_or_raise  40  : The following links have carriers which are not defined:
Index(['IT0 0 H2 Electrolysis', 'IT0 0 H2 Fuel Cell'], dtype='object', name='Link')
WARNING    2025-09-17 15:08:44,214 pypsa.consistency _log_or_raise  40  : The following stores have carriers which are not defined:
Index(['IT0 0 H2'], dtype='object', name='Store')
INFO       2025-09-17 15:08:45,788 linopy.model solve       114

Set parameter Username


INFO       2025-09-17 15:08:46,102 gurobipy   solve_problem_from_file  1033: Set parameter Username


Set parameter LicenseID to value 2647006


INFO       2025-09-17 15:08:46,104 gurobipy   solve_problem_from_file  1033: Set parameter LicenseID to value 2647006


...

Solved in 18 iterations and 0.04 seconds (0.00 work units)


INFO       2025-09-17 15:08:46,167 gurobipy   _solve      1113: Solved in 18 iterations and 0.04 seconds (0.00 work units)


Optimal objective  4.139885957e+08


INFO       2025-09-17 15:08:46,168 gurobipy   _solve      1113: Optimal objective  4.139885957e+08
INFO       2025-09-17 15:08:46,178 linopy.constants info        246 :  Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 170 primals, 362 duals
Objective: 4.14e+08
Solver model: available
Solver message: 2

INFO       2025-09-17 15:08:46,298 pypsa.optimization.optimize assign_duals  681 : The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Link-fix-p-lower, Link-fix-p-upper, Store-fix-e-lower, Store-fix-e-upper, Store-energy_balance were not assigned to the network.
/dati/pampado/pypsa2smspp/pypsa2smspp/transformation_config.py:90: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/dati/pampado/pypsa2smsp

La classe di trasformazione ci mette 0:00:00.190031 secondi


In [2]:
a = 5
if transformation.dimensions['InvestmentBlock']['NumAssets'] == 0 or a == 5:
    ### UCBlock configuration ###
    configfile = pysmspp.SMSConfig(template="UCBlock/uc_solverconfig_grb")  # load a default config file [highs solver]
    temporary_smspp_file = "output/network_uc_hydro_0011.nc"  # path to temporary SMS++ file
    output_file = "output/temp_log_file.txt"  # path to the output file (optional)
    solution_file = "output/solution_uc_hydro_0011.nc"
    
    # Check if the file exists
    if os.path.exists(solution_file):
        os.remove(solution_file)
    
    result = tran.optimize(configfile, temporary_smspp_file, output_file, solution_file, log_executable_call=True)
    
    statistics = network.statistics()
    operational_cost = statistics['Operational Expenditure'].sum()
    # error = (operational_cost - result.objective_value) / operational_cost * 100

    objective_pypsa = network.objective # + network.objective_constant
    objective_smspp = result.objective_value
    error = (objective_pypsa - objective_smspp) / objective_pypsa * 100
    
    print(f"Error PyPSA-SMS++ of {error}%")
    
    # Esegui la funzione sul file di testo
    data_dict = parse_txt_file(output_file)

    print(f"Il solver ci ha messo {data_dict['elapsed_time']}s")
    print(f"Il tempo totale (trasformazione+pysmspp+ottimizzazione smspp) è {datetime.now() - then}")

    
    solution = transformation.parse_solution_to_unitblocks(result.solution, nd.n)
    # transformation.parse_txt_to_unitblocks(output_file)
    transformation.inverse_transformation(nd.n)

    differences = compare_networks(network, nd.n)
    statistics_smspp = nd.n.statistics()
    

else:
    ### InvestmentBlock configuration ###
    configfile = pysmspp.SMSConfig(template="InvestmentBlock/BSPar.txt")
    temporary_smspp_file = "output/temp_network_investment_daily.nc"
    output_file = "output/temp_log_file_investment.txt"  # path to the output file (optional)
    solution_file = "output/temp_solution_file_investment.nc"
    
    # Check if the file exists
    if os.path.exists(solution_file):
        os.remove(solution_file)
    
    result = tran.optimize(configfile, temporary_smspp_file, output_file, solution_file, inner_block_name='InvestmentBlock', log_executable_call=True)
    
    
    objective_pypsa = network.objective # + network.objective_constant
    objective_smspp = result.objective_value
    error = (objective_pypsa - objective_smspp) / objective_pypsa * 100
    
    print(f"Error PyPSA-SMS++ of {error}%")
    print(f"Il tempo totale (trasformazione+pysmspp+ottimizzazione smspp) è {datetime.now() - then}")

    solution = transformation.parse_solution_to_unitblocks(result.solution, nd.n)
    transformation.inverse_transformation(nd.n)

Executing command:
ucblock_solver /dati/pampado/pypsa2smspp/test/output/network_uc_hydro_0011.nc -c /dati/pampado/pySMSpp/pysmspp/data/configs/UCBlock/ -S uc_solverconfig_grb.txt -o -O /dati/pampado/pypsa2smspp/test/output/solution_uc_hydro_0011.nc

Error PyPSA-SMS++ of -54.40703020934829%
Il solver ci ha messo 0.003524296s
Il tempo totale (trasformazione+pysmspp+ottimizzazione smspp) è 0:00:00.974319
[split] 'DCNetworkBlock_4' -> 'IT0 0 H2 Electrolysis' + 'IT0 0 H2 Fuel Cell'


In [3]:
network.statistics()

Optimal Capacity  Installed Capacity       Supply  \
Generator slack                 13179.69395         13179.69395  41398.83402   
          solar                 19711.86918         19711.86918  25544.78232   
Link      H2 electrolysis         100.00000           100.00000      0.00000   
          H2 fuel cell            100.00000           100.00000      0.00000   
Load      AC                        0.00000             0.00000      0.00000   
Store     H2                      635.00000           635.00000    635.00000   

                            Withdrawal  Energy Balance Transmission  \
Generator slack                0.00000     41398.83402          0.0   
          solar                0.00000     25544.78232          0.0   
Link      H2 electrolysis   1099.94803     -1099.94803          0.0   
          H2 fuel cell       635.00000      -635.00000          0.0   
Load      AC               66161.16831    -66161.16831          0.0   
Store     H2                 635.00000         0.00000          0.0   

                           Capacity Factor   Curtailment  Capital Expenditure  \
Generator slack                   0.130879  274913.82075         0.000000e+00   
          solar                   0.053996  101123.68905         1.079110e+09   
Link      H2 electrolysis         0.458312       0.00000         2.516210e+07   
          H2 fuel cell            0.264583       0.00000         1.323257e+07   
Load      AC                           NaN       0.00000         0.000000e+00   
Store     H2                      0.403862       0.00000         1.412782e+05   

                           Operational Expenditure       Revenue  Market Value  
Generator slack                       4.139883e+08  4.139883e+08   9999.999986  
          solar                       2.554478e+02  7.293640e+06    285.523660  
Link      H2 electrolysis             0.000000e+00 -1.099948e+01           NaN  
          H2 fuel cell                0.000000e+00 -3.175000e+06           NaN  
Load      AC                          0.000000e+00 -4.244570e+08           NaN  
Store     H2                          0.000000e+00  3.174989e+06   4999.983308

In [4]:
nd.n.statistics()

Optimal Capacity  Installed Capacity       Supply  \
Generator slack                 13179.69395         13179.69395  46880.03867   
          solar                 19711.86918         19711.86918  19371.47110   
Link      H2 electrolysis         100.00000           100.00000      0.00000   
          H2 fuel cell            100.00000           100.00000      0.00000   
Load      AC                        0.00000             0.00000      0.00000   
Store     H2                      635.00000           635.00000      0.00000   

                            Withdrawal  Energy Balance Transmission  \
Generator slack                0.00000     46880.03867          0.0   
          solar                0.00000     19371.47110          0.0   
Link      H2 electrolysis    127.00000      -127.00000          0.0   
          H2 fuel cell      1526.90711     -1526.90711          0.0   
Load      AC               66161.16831    -66161.16831          0.0   
Store     H2                   0.00000         0.00000          0.0   

                           Capacity Factor   Curtailment  Capital Expenditure  \
Generator slack                   0.148208  269432.61611         0.000000e+00   
          solar                   0.040947  107297.00028         1.079110e+09   
Link      H2 electrolysis         0.052917       0.00000         2.516210e+07   
          H2 fuel cell            0.636211       0.00000         1.323257e+07   
Load      AC                           NaN       0.00000         0.000000e+00   
Store     H2                      0.000000       0.00000         1.412782e+05   

                           Operational Expenditure  Revenue  Market Value  
Generator slack                       4.688004e+08      0.0           NaN  
          solar                       1.937147e+02      0.0           NaN  
Link      H2 electrolysis             0.000000e+00      0.0           0.0  
          H2 fuel cell                0.000000e+00      0.0           0.0  
Load      AC                          0.000000e+00      0.0           0.0  
Store     H2                          0.000000e+00      0.0           0.0

In [10]:
a = transformation.unitblocks['BatteryUnitBlock_3']['variables']['ConverterMaxPower']

In [27]:
network.objective / 1e8

4.139885956833195

In [25]:
network.links_t['p0']

Link,IT0 0 H2 Electrolysis,IT0 0 H2 Fuel Cell
snapshot,,
0,0.000000,0.0
1,0.000000,0.0
2,0.000000,0.0
3,0.000000,0.0
4,99.948034,0.0
5,100.000000,0.0
6,100.000000,0.0
7,100.000000,0.0
8,100.000000,0.0
